# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,273.76,64,75,9.77,CA,1.682610e+09
1,1,xianning,29.8833,114.2167,291.49,77,65,0.79,CN,1.682610e+09
2,2,edinburgh of the seven seas,-37.0676,-12.3116,290.43,77,97,8.64,SH,1.682610e+09
3,3,yellowknife,62.4560,-114.3525,273.46,100,100,4.63,CA,1.682610e+09
4,4,aleksandrovsk-sakhalinskiy,50.8975,142.1561,273.40,99,100,1.61,RU,1.682610e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM"
)

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_city_data_df = city_data_df.loc[
    (city_data_df["Humidity"] >= 50) & 
    (city_data_df["Humidity"] < 85) &
    (city_data_df["Cloudiness"] >= 40) &
    (city_data_df["Wind Speed"] < 8)
]

# Drop any rows with null values
vacation_city_data_df.dropna(axis=0, how="any", inplace=True)

# Display sample data
vacation_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,xianning,29.8833,114.2167,291.49,77,65,0.79,CN,1.682610e+09
11,11,port mathurin,-19.6833,63.4167,299.41,80,42,6.79,MU,1.682610e+09
12,12,keflavik,64.0049,-22.5624,277.39,81,75,5.14,IS,1.682610e+09
15,15,happy valley-goose bay,53.3168,-60.3315,279.32,65,40,2.06,CA,1.682610e+09
19,19,kapuskasing,49.4169,-82.4331,277.15,70,100,3.09,CA,1.682610e+09
...,...,...,...,...,...,...,...,...,...,...
531,531,svoge,42.9667,23.3500,284.54,75,100,1.91,BG,1.682610e+09
532,532,red oak,32.5176,-96.8044,291.85,75,75,6.17,US,1.682610e+09
533,533,ta`u,-14.2336,-169.5144,300.90,73,100,4.37,AS,1.682610e+09
540,540,brigantine,39.4101,-74.3646,287.06,79,100,5.14,US,1.682610e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(9, "Hotel Name", "")
hotel_df = hotel_df.drop(columns=["City_ID", "Max Temp", "Date"])

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
1,xianning,29.8833,114.2167,77,65,0.79,CN,
11,port mathurin,-19.6833,63.4167,80,42,6.79,MU,
12,keflavik,64.0049,-22.5624,81,75,5.14,IS,
15,happy valley-goose bay,53.3168,-60.3315,65,40,2.06,CA,
19,kapuskasing,49.4169,-82.4331,70,100,3.09,CA,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 100
params = {
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
xianning - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
keflavik - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: No hotel found
kapuskasing - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
san antonio la paz - nearest hotel: No hotel found
jining - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
margate - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
coquimbo - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
ust'-kulom - nearest hotel: No hotel found
balaklava - nearest hotel: No hotel found
filadelfia - nearest hotel: No hotel found
cururupu - nearest hotel: No hotel found
tibati - nearest hotel: No hotel found
college - nearest hotel: No hotel found

,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
1,xianning,29.8833,114.2167,77,65,0.79,CN,No hotel found
11,port mathurin,-19.6833,63.4167,80,42,6.79,MU,No hotel found
12,keflavik,64.0049,-22.5624,81,75,5.14,IS,No hotel found
15,happy valley-goose bay,53.3168,-60.3315,65,40,2.06,CA,No hotel found
19,kapuskasing,49.4169,-82.4331,70,100,3.09,CA,No hotel found
...,...,...,...,...,...,...,...,...
531,svoge,42.9667,23.3500,75,100,1.91,BG,No hotel found
532,red oak,32.5176,-96.8044,75,75,6.17,US,No hotel found
533,ta`u,-14.2336,-169.5144,73,100,4.37,AS,No hotel found
540,brigantine,39.4101,-74.3646,79,100,5.14,US,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)